# California Housing - Lineare Regression

Dieses Notebook führt euch durch eine komplette Machine Learning Pipeline mit dem California Housing Datensatz.

## Was wir machen:
1. **Daten laden** und verstehen
2. **Train/Test Split** für ehrliche Evaluierung  
3. **Lineares Modell** trainieren
4. **Performance bewerten** mit Metriken
5. **Ergebnisse visualisieren** mit Grafiken

In [ ]:
# Schritt 1: Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

print("Alle Bibliotheken erfolgreich importiert! 🎉")

In [ ]:
# Schritt 2: Daten laden
print("Lade California Housing Datensatz...")
california = fetch_california_housing()

# Als pandas DataFrame für bessere Handhabung
X = pd.DataFrame(california.data, columns=california.feature_names)
y = pd.Series(california.target, name='HouseValue')

print(f"✅ Datensatz geladen: {X.shape[0]} Regionen, {X.shape[1]} Features")
print(f"📊 Features: {list(california.feature_names)}")

In [ ]:
# Schritt 3: Daten verstehen
print("🔍 Erste 5 Datensätze:")
display(X.head())

print(f"\n💰 Hauspreise - Erste 5 Werte:")
print(y.head().values)

print(f"\n📈 Preis-Statistiken:")
print(f"   Durchschnitt: ${y.mean():.2f} (×100k) = ${y.mean()*100:.0f}k")
print(f"   Minimum: ${y.min():.2f} (×100k) = ${y.min()*100:.0f}k")  
print(f"   Maximum: ${y.max():.2f} (×100k) = ${y.max()*100:.0f}k")

In [ ]:
# Schritt 3b: Dataset-Eigenart verstehen - Preiskappung bei $500k
print("🚨 WICHTIGER HINWEIS - Dataset-Eigenart:")
print(f"   Maximum im Dataset: ${y.max():.5f} (×100k) = ${y.max()*100:.0f}k")
print(f"   Werte bei genau $500k: {np.sum(y == 5.0)} von {len(y)} ({np.sum(y == 5.0)/len(y)*100:.1f}%)")
print(f"   Werte ≥ $490k: {np.sum(y >= 4.9)} von {len(y)} ({np.sum(y >= 4.9)/len(y)*100:.1f}%)")

# Visualisierung des Kappungseffekts
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(y, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(5.0, color='red', linestyle='--', linewidth=2, label='Kappung bei $500k')
plt.xlabel('Hauswerte (×100k $)')
plt.ylabel('Häufigkeit')
plt.title('Verteilung der Hauswerte mit Kappung')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(y[y >= 4.5], bins=20, alpha=0.7, edgecolor='black', color='orange')
plt.axvline(5.0, color='red', linestyle='--', linewidth=2, label='Kappung bei $500k')
plt.xlabel('Hauswerte (×100k $)')
plt.ylabel('Häufigkeit')
plt.title('Zoom: Teure Häuser (≥$450k)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📚 QUELLEN & HINTERGRUND:")
print(f"   📄 Original-Studie: Pace & Barry (1997), Statistics and Probability Letters")
print(f"   🏛️ Datenquelle: US Census 1990, California Block Groups")
print(f"   ⚙️ Preprocessing: Häuser >$500k wurden auf genau $500k 'gekappt'")
print(f"   🎯 Grund: Datenschutz & Ausreißer-Reduktion in der Originalerhebung")
print(f"   📊 Effekt: Künstliche Häufung bei $500k verfälscht teure Vorhersagen!")


In [ ]:
# Schritt 4: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% für Test
    random_state=42     # Für reproduzierbare Ergebnisse
)

print(f"🚂 Training: {X_train.shape[0]} Regionen")
print(f"🧪 Test: {X_test.shape[0]} Regionen")
print(f"⚖️ Verhältnis: {X_train.shape[0]/X_test.shape[0]:.1f}:1 (Train:Test)")

In [ ]:
# Schritt 5: Modell trainieren
model = LinearRegression()

print("🤖 Trainiere das Modell...")
model.fit(X_train, y_train)

print(f"✅ Modell trainiert!")
print(f"🏠 Basis-Hauspreis: ${model.intercept_:.2f} (×100k)")

# Die wichtigsten Features zeigen
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Koeffizient': model.coef_
}).sort_values('Koeffizient', key=abs, ascending=False)

print(f"\n⭐ Top 5 wichtigste Features:")
for i, (_, row) in enumerate(feature_importance.head().iterrows(), 1):
    print(f"   {i}. {row['Feature']}: {row['Koeffizient']:.3f}")

In [ ]:
# Schritt 6: Vorhersagen und Bewertung
y_pred = model.predict(X_test)

# Performance-Metriken berechnen
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"📊 Modell-Performance:")
print(f"   RMSE: ${rmse:.2f} (×100k) = ${rmse*100:.0f}k Durchschnittsfehler")  
print(f"   R²-Score: {r2:.3f} = {r2*100:.1f}% der Variation erklärt")

# Beispiel-Vorhersagen zeigen
print(f"\n🎯 Beispiel-Vorhersagen:")
for i in range(5):
    actual = y_test.iloc[i] * 100  # In normale Tausend Dollar umrechnen
    predicted = y_pred[i] * 100
    error = abs(actual - predicted)
    print(f"   Tatsächlich: ${actual:.0f}k, Vorhersage: ${predicted:.0f}k, Fehler: ${error:.0f}k")

In [ ]:
# Schritt 7: Ergebnisse visualisieren
plt.figure(figsize=(12, 4))

# Grafik 1: Vorhersage vs. Realität
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.6, s=30)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Ideale Linie (y = ŷ)')
plt.xlabel('Tatsächliche Preise (×100k $)')
plt.ylabel('Vorhergesagte Preise (×100k $)')
plt.title('Vorhersage vs. Realität')
plt.legend()
plt.grid(True, alpha=0.3)

# Grafik 2: Fehler-Verteilung
plt.subplot(1, 2, 2)
errors = (y_test - y_pred) * 100  # In Tausend Dollar
plt.hist(errors, bins=30, alpha=0.7, edgecolor='black', color='orange')
plt.xlabel('Vorhersagefehler (Tausend $)')
plt.ylabel('Häufigkeit')
plt.title('Verteilung der Vorhersagefehler')
plt.grid(True, alpha=0.3)
plt.axvline(0, color='red', linestyle='--', alpha=0.8, label='Kein Fehler')
plt.legend()

plt.tight_layout()
plt.show()

print(f"📈 Durchschnittlicher Absolut-Fehler: ${np.mean(np.abs(errors)):.0f}k")

## 🎉 Geschafft!

**Was wir erreicht haben:**
- ✅ Komplette ML-Pipeline von Daten bis Vorhersage
- ✅ R²-Score von ~0.60-0.67 (60-67% der Variation erklärt)
- ✅ Durchschnittsfehler von ~65-75k Dollar bei Hauspreisen

**Was wir gelernt haben:**
- 🏠 **MedInc** (Einkommen) ist der wichtigste Faktor für Hauspreise
- 📍 **Lage** (Latitude/Longitude) ist auch sehr wichtig
- 🔢 **Lineare Regression** ist ein guter Startpunkt für Preisvorhersagen
- 📊 **Train/Test Split** ist essentiell für ehrliche Modell-Bewertung

**⚠️ Wichtige Dataset-Eigenart:**
- **Preiskappung bei $500k**: Das Dataset wurde in der Originalerhebung (US Census 1990) bei $500k gekappt
- **Quellen**: Pace & Barry (1997), "Sparse Spatial Autoregressions", Statistics and Probability Letters
- **Effekt auf Vorhersagen**: 
  - Modelle können keine Häuser >$500k korrekt vorhersagen
  - Künstliche Häufung bei $500k verfälscht teure Immobilien-Prognosen
  - In der Realität würden moderne Datasets höhere Preise enthalten

**Nächste Schritte:**
- Probiert andere Algorithmen aus (Random Forest, XGBoost)
- Feature Engineering: Neue Features aus bestehenden ableiten
- Hyperparameter-Tuning für bessere Performance
- **Moderne Datasets**: Verwende aktuellere Immobiliendaten ohne Kappung!